# Photoredox Yield Noise Detection Benchmark

## Goal
Compare **Old GGH (DBSCAN)** vs **New GGH (Soft Refinement)** for noise detection.

## Challenge
- **Unsupervised**: No labeled clean samples available during detection
- **Simulated noise**: 30% of target values corrupted (range 0.4-0.6 around mean)
- **Evaluation only**: Labels used only to measure detection performance

## Methods Compared
1. **Full Info (No Noise)**: Oracle upper bound
2. **Full Info Noisy**: Baseline with noise, no removal
3. **Old GGH (DBSCAN)**: Existing unsupervised clustering approach
4. **New GGH (Soft Refinement)**: Bootstrap anchors from data, iterative refinement

## Expected Results
Based on old notebooks:
- Full Info: R2 ~0.858
- Full Info Noisy: R2 ~0.608
- Old GGH: R2 ~0.674 (improvement of 0.066)
- **Target**: New GGH R2 > 0.70

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from tqdm import tqdm
import sys
import math
from copy import deepcopy
sys.path.insert(0, '../')
sys.path.insert(0, '../GGH')

from GGH.data_ops import DataOperator
from GGH.selection_algorithms import AlgoModulators
from GGH.models import initialize_model
from GGH.train_val_loop import TrainValidationManager
from GGH.inspector import Inspector, get_gradarrays_n_labels
from scipy import stats
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import grad
from sklearn.cluster import DBSCAN
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

def set_to_deterministic(rand_state):
    import random
    random.seed(rand_state)
    np.random.seed(rand_state)
    torch.manual_seed(rand_state)
    torch.set_num_threads(1)
    torch.use_deterministic_algorithms(True)

print("Imports successful!")

# GPU Detection
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")


In [ ]:
# =============================================================================
# CONFIGURATION - Photoredox Yield Dataset
# =============================================================================
data_path = '../data/photoredox_yield/photo_redox_merck2021_1649reactions.csv'
results_path = "../saved_results/Photoredox Yield Noise Detection Benchmark"

# Variables
inpt_vars = ['aryl_halides', 'photocalysts', 'piperidines_moles', 'photocalysts_moles']
target_vars = ['uplcms']
miss_vars = []
hypothesis = [[0.02, 0.05, 0.50, 5.0]]

# Model parameters
hidden_size = 32
output_size = len(target_vars)

# Noise simulation parameters
DATA_NOISE_PERC = 0.30  # 30% of data will have noise
NOISE_MINRANGE = 0.40   # Noise factor range
NOISE_MAXRANGE = 0.60
noise_profile = {
    "DATA_NOISE_PERC": DATA_NOISE_PERC,
    "NOISE_MINRANGE": NOISE_MINRANGE,
    "NOISE_MAXRANGE": NOISE_MAXRANGE
}

# Training parameters
partial_perc = 0.3  # Not used for noise detection, but required by DataOperator
batch_size = 299
dropout = 0.05
lr = 0.001
nu = 0.1

# Benchmark parameters
BENCHMARK_N_RUNS = 15
FULL_INFO_EPOCHS = 300
NOISY_EPOCHS = 300

# New GGH parameters
GGH_ITER1_EPOCHS = 60
GGH_ITER1_ANALYSIS_EPOCHS = 5
GGH_ITER2_EPOCHS = 30
GGH_FINAL_EPOCHS = 300
GGH_MIN_WEIGHT = 0.1
GGH_TEMPERATURE = 1.0
GGH_NOISE_THRESHOLD = 0.3
GGH_CLEAN_PERCENTILE = 0.60  # Bottom 60% by loss = likely clean

# Old GGH (DBSCAN) parameters
OLD_GGH_EPOCHS = 200
OLD_GGH_END_EPOCHS = 10
OLD_GGH_EPS_VALUES = [0.15, 0.25]
OLD_GGH_MIN_SAMPLES_RATIOS = [0.15, 0.2, 0.25]

# Create directories
import os
os.makedirs(results_path, exist_ok=True)

print(f"Dataset: Photoredox Yield")
print(f"Noise simulation: {DATA_NOISE_PERC*100}% of data, range [{NOISE_MINRANGE}, {NOISE_MAXRANGE}]")
print(f"Benchmark runs: {BENCHMARK_N_RUNS}")
print(f"Results path: {results_path}")

## Helper Functions

Shared utility functions for both old and new GGH.

In [ ]:
def sigmoid_stable(x):
    """Numerically stable sigmoid."""
    x = np.array(x, dtype=np.float64)
    return np.where(x >= 0,
                    1 / (1 + np.exp(-x)),
                    np.exp(x) / (1 + np.exp(x)))

def compute_soft_weights(scores, min_weight=0.1, temperature=1.0):
    """Convert scores to soft weights using sigmoid."""
    scores = np.array(scores, dtype=np.float64)
    if len(scores) == 0:
        return np.array([])
    
    mean_s = np.mean(scores)
    std_s = np.std(scores) + 1e-8
    normalized = (scores - mean_s) / std_s
    
    raw_weights = sigmoid_stable(normalized / temperature)
    weights = min_weight + (1 - min_weight) * raw_weights
    
    return weights

def evaluate_detection(detected_noisy_indices, true_noisy_indices, n_total):
    """Evaluate noise detection performance."""
    detected_set = set(detected_noisy_indices)
    true_noisy_set = set(true_noisy_indices)
    true_clean_set = set(range(n_total)) - true_noisy_set
    
    TP = len(detected_set & true_noisy_set)  # Correctly detected noisy
    FP = len(detected_set & true_clean_set)  # Clean wrongly labeled as noisy
    FN = len(true_noisy_set - detected_set)  # Noisy missed
    TN = len(true_clean_set - detected_set)  # Clean correctly kept
    
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    accuracy = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        'precision': precision,
        'recall': recall,
        'accuracy': accuracy,
        'f1': f1,
        'TP': TP,
        'FP': FP,
        'FN': FN,
        'TN': TN
    }

def full_info_run_existing_DO(DO, df_train_cleaned, r_state, num_epochs=300):
    """Train model on cleaned data (after noise removal)."""
    use_info = "full info"
    AM = AlgoModulators(DO, lr=lr)
    DO_clean = DataOperator(data_path, inpt_vars, target_vars, miss_vars, hypothesis, 
                            partial_perc, r_state, pre_defined_train=df_train_cleaned, 
                            device="cpu")
    dataloader = DO_clean.prep_dataloader(use_info, batch_size)
    model = initialize_model(DO_clean, dataloader, hidden_size, r_state, dropout=dropout)
    TVM = TrainValidationManager(use_info, num_epochs, dataloader, batch_size, r_state, 
                                 results_path, best_valid_error=np.inf)
    TVM.train_model(DO_clean, AM, model, final_analysis=False)
    
    return DO_clean, TVM, AM, model

print("Helper functions defined.")

## Old GGH: DBSCAN-based Noise Detection

Original approach from the noise detection notebooks:
1. Train on noisy data
2. Extract gradients
3. Use DBSCAN clustering - outliers = noisy
4. Grid search over eps and min_samples

In [ ]:
def run_old_ggh_dbscan(DO, r_state):
    """
    Old GGH noise detection using DBSCAN clustering.
    Returns best result from hyperparameter grid search.
    """
    use_info = "full info noisy"
    num_epochs = OLD_GGH_EPOCHS + OLD_GGH_END_EPOCHS
    
    best_result = None
    best_val_error = np.inf
    
    for eps in OLD_GGH_EPS_VALUES:
        for min_samples_ratio in OLD_GGH_MIN_SAMPLES_RATIOS:
            set_to_deterministic(r_state)
            
            # Train on noisy data
            AM = AlgoModulators(DO, lr=lr, eps_value=eps, min_samples_ratio=min_samples_ratio)
            dataloader = DO.prep_dataloader(use_info, batch_size)
            model = initialize_model(DO, dataloader, hidden_size, r_state, dropout=dropout)
            
            TVM = TrainValidationManager(use_info, num_epochs, dataloader, batch_size, r_state,
                                        results_path, select_gradients=True,
                                        end_epochs_noise_detection=OLD_GGH_END_EPOCHS,
                                        best_valid_error=np.inf, final_analysis=False)
            TVM.train_model(DO, AM, model, final_analysis=False)
            
            # Extract gradients
            array_grads_context, do_hyp_class = get_gradarrays_n_labels(
                DO, 0, layer=-2, remov_avg=False, include_context=False,
                normalize_grads_context=False, loss_in_context=True, only_loss_context=True,
                num_batches=math.ceil(len(DO.df_train_noisy) / batch_size),
                epoch=TVM.best_checkpoint, use_case="noise_detection"
            )
            
            # DBSCAN clustering
            dbscan = DBSCAN(eps=eps, min_samples=int(batch_size * min_samples_ratio))
            pred_labels = dbscan.fit_predict(array_grads_context)
            pred_labels = pred_labels * -1  # Invert: 1=noisy (outliers), 0=clean
            
            # Detect noisy samples (label == 1)
            detected_noisy_indices = [i for i, label in enumerate(pred_labels) if label == 1]
            
            # Remove detected noisy and retrain
            DO.df_train_noisy["noise_detected"] = pred_labels
            df_cleaned = deepcopy(DO.df_train_noisy[DO.df_train_noisy["noise_detected"] == 0])
            
            DO_clean, TVM_clean, AM_clean, model_clean = full_info_run_existing_DO(
                DO, df_cleaned, r_state, num_epochs=GGH_FINAL_EPOCHS
            )
            
            # Keep best hyperparameters
            if TVM_clean.best_valid_error < best_val_error:
                best_val_error = TVM_clean.best_valid_error
                
                # Evaluate detection
                true_noisy_indices = DO.df_train_noisy[DO.df_train_noisy['label'] == 1].index.tolist()
                detection_metrics = evaluate_detection(
                    detected_noisy_indices, true_noisy_indices, len(DO.df_train_noisy)
                )
                
                # Test performance
                model_clean.eval()
                with torch.no_grad():
                    test_inputs, test_targets = DO.get_test_tensors(use_info="full info")
                    test_preds = model_clean(test_inputs)
                    ss_res = torch.sum((test_targets - test_preds) ** 2).item()
                    ss_tot = torch.sum((test_targets - test_targets.mean()) ** 2).item()
                    test_r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0.0
                    test_mse = torch.nn.functional.mse_loss(test_preds, test_targets).item()
                    test_mae = torch.nn.functional.l1_loss(test_preds, test_targets).item()
                
                best_result = {
                    'eps': eps,
                    'min_samples_ratio': min_samples_ratio,
                    'test_r2': test_r2,
                    'test_mse': test_mse,
                    'test_mae': test_mae,
                    'detection': detection_metrics,
                    'n_detected': len(detected_noisy_indices),
                    'n_removed': len(detected_noisy_indices),
                }
    
    return best_result

print("Old GGH (DBSCAN) function defined.")

## New GGH: Unsupervised Soft Refinement for Noise Detection

Bootstrap approach:
1. **Iter1**: Train unbiased → bootstrap clean/noisy anchors from loss distribution
2. **Iter2**: Weighted training (high weight = likely clean)
3. **Iter3**: Refined anchors → rescore samples
4. **Detection**: Threshold on final weights

In [ ]:
class SimpleNoiseDetectionModel(nn.Module):
    """Simple MLP for noise detection."""
    def __init__(self, n_features, hidden_size=32, output_size=1):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(n_features, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )
    
    def forward(self, x):
        return self.network(x)

print("Model defined.")

In [ ]:
def run_new_ggh_unsupervised(DO, r_state):
    """
    New GGH unsupervised noise detection using soft refinement.
    
    Bootstraps clean/noisy anchors from loss distribution, then iteratively refines.
    NO LABELS USED during detection - fully unsupervised.
    """
    n_features = len(inpt_vars)
    n_samples = len(DO.df_train_noisy)
    
    # Create dataloader
    train_features = torch.tensor(DO.df_train_noisy[inpt_vars].values, dtype=torch.float32)
    train_targets = torch.tensor(DO.df_train_noisy[target_vars].values, dtype=torch.float32)
    sample_indices = torch.arange(n_samples)
    dataset = TensorDataset(train_features, train_targets, sample_indices)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # =============================================================================
    # ITERATION 1: Unbiased training + Bootstrap anchors from loss distribution
    # =============================================================================
    print("  Iter1: Unbiased training + bootstrap anchors...")
    set_to_deterministic(r_state)
    model_iter1 = SimpleNoiseDetectionModel(n_features, hidden_size, output_size)
    optimizer = torch.optim.Adam(model_iter1.parameters(), lr=lr)
    criterion = nn.MSELoss(reduction='none')
    
    # Track losses and gradients
    sample_losses = {i: [] for i in range(n_samples)}
    sample_gradients = {i: [] for i in range(n_samples)}
    
    # Train for total epochs, track during last few
    total_iter1_epochs = GGH_ITER1_EPOCHS
    track_start = GGH_ITER1_EPOCHS - GGH_ITER1_ANALYSIS_EPOCHS
    
    for epoch in range(total_iter1_epochs):
        model_iter1.train()
        track_this_epoch = (epoch >= track_start)
        
        for features, targets, indices in dataloader:
            predictions = model_iter1(features)
            losses = criterion(predictions, targets).squeeze()
            batch_loss = losses.mean()
            
            if track_this_epoch:
                # Track per-sample losses
                for i, idx in enumerate(indices):
                    sample_losses[idx.item()].append(losses[i].item())
                
                # Track per-sample gradients
                for i, idx in enumerate(indices):
                    feat = features[i:i+1].clone().requires_grad_(True)
                    pred = model_iter1(feat)
                    loss = criterion(pred, targets[i:i+1]).mean()
                    
                    params = list(model_iter1.parameters())
                    grad_param = grad(loss, params[-2], retain_graph=False)[0]
                    grad_vec = grad_param.flatten().detach().cpu().numpy()
                    sample_gradients[idx.item()].append(grad_vec)
            
            optimizer.zero_grad()
            batch_loss.backward()
            optimizer.step()
    
    # Compute average losses and gradients
    avg_losses = {i: np.mean(sample_losses[i]) for i in range(n_samples) if sample_losses[i]}
    avg_gradients = {i: np.mean(sample_gradients[i], axis=0) for i in range(n_samples) if sample_gradients[i]}
    
    # Bootstrap clean/noisy candidates from loss distribution (UNSUPERVISED)
    loss_values = np.array([avg_losses[i] for i in range(n_samples)])
    loss_threshold = np.percentile(loss_values, GGH_CLEAN_PERCENTILE * 100)
    
    clean_candidates = [i for i in range(n_samples) if avg_losses[i] <= loss_threshold]
    noisy_candidates = [i for i in range(n_samples) if avg_losses[i] > loss_threshold]
    
    print(f"    Bootstrapped: {len(clean_candidates)} clean, {len(noisy_candidates)} noisy candidates")
    
    # Build initial anchors
    if clean_candidates and noisy_candidates:
        clean_anchor_grad = np.mean([avg_gradients[i] for i in clean_candidates if i in avg_gradients], axis=0)
        noisy_anchor_grad = np.mean([avg_gradients[i] for i in noisy_candidates if i in avg_gradients], axis=0)
        
        # Score all samples
        sample_scores = {}
        for i in range(n_samples):
            if i not in avg_gradients:
                sample_scores[i] = 0.0
                continue
            
            grad = avg_gradients[i]
            sim_clean = np.dot(grad, clean_anchor_grad) / (np.linalg.norm(grad) * np.linalg.norm(clean_anchor_grad) + 1e-8)
            sim_noisy = np.dot(grad, noisy_anchor_grad) / (np.linalg.norm(grad) * np.linalg.norm(noisy_anchor_grad) + 1e-8)
            sample_scores[i] = float(sim_clean - sim_noisy)
        
        # Convert to soft weights
        scores_list = [sample_scores[i] for i in range(n_samples)]
        sample_weights = compute_soft_weights(scores_list, GGH_MIN_WEIGHT, GGH_TEMPERATURE)
        sample_weights_dict = {i: float(sample_weights[i]) for i in range(n_samples)}
    else:
        print("    Warning: Could not bootstrap anchors, using uniform weights")
        sample_weights_dict = {i: 0.5 for i in range(n_samples)}
    
    avg_weight = np.mean(list(sample_weights_dict.values()))
    print(f"    Avg weight: {avg_weight:.3f}")
    
    # =============================================================================
    # ITERATION 2: Weighted training (high weight = likely clean)
    # =============================================================================
    print("  Iter2: Weighted training...")
    set_to_deterministic(r_state + 100)
    model_iter2 = SimpleNoiseDetectionModel(n_features, hidden_size, output_size)
    optimizer2 = torch.optim.Adam(model_iter2.parameters(), lr=lr)
    
    for epoch in range(GGH_ITER2_EPOCHS):
        model_iter2.train()
        for features, targets, indices in dataloader:
            predictions = model_iter2(features)
            losses = criterion(predictions, targets).squeeze()
            
            # Weight by Iter1 scores
            weights = torch.tensor([sample_weights_dict[idx.item()] for idx in indices], dtype=torch.float32)
            weighted_loss = (losses * weights).sum() / weights.sum()
            
            optimizer2.zero_grad()
            weighted_loss.backward()
            optimizer2.step()
    
    # =============================================================================
    # ITERATION 3: Refined anchors from biased model
    # =============================================================================
    print("  Iter3: Refined anchors + rescoring...")
    model_iter2.eval()
    
    # Recompute losses and gradients with biased model
    iter3_losses = {i: [] for i in range(n_samples)}
    iter3_gradients = {i: [] for i in range(n_samples)}
    
    for _ in range(3):  # Multiple passes for stability
        for features, targets, indices in dataloader:
            for i, idx in enumerate(indices):
                feat = features[i:i+1].clone().requires_grad_(True)
                pred = model_iter2(feat)
                loss = criterion(pred, targets[i:i+1]).mean()
                
                iter3_losses[idx.item()].append(loss.item())
                
                params = list(model_iter2.parameters())
                grad_param = grad(loss, params[-2], retain_graph=False)[0]
                grad_vec = grad_param.flatten().detach().cpu().numpy()
                iter3_gradients[idx.item()].append(grad_vec)
    
    avg_iter3_losses = {i: np.mean(iter3_losses[i]) for i in range(n_samples) if iter3_losses[i]}
    avg_iter3_gradients = {i: np.mean(iter3_gradients[i], axis=0) for i in range(n_samples) if iter3_gradients[i]}
    
    # Refined anchors: use top-weighted samples from Iter1
    weight_threshold_top = np.percentile(list(sample_weights_dict.values()), 70)
    weight_threshold_bottom = np.percentile(list(sample_weights_dict.values()), 30)
    
    refined_clean = [i for i in range(n_samples) if sample_weights_dict[i] >= weight_threshold_top]
    refined_noisy = [i for i in range(n_samples) if sample_weights_dict[i] <= weight_threshold_bottom]
    
    if refined_clean and refined_noisy:
        refined_clean_anchor = np.mean([avg_iter3_gradients[i] for i in refined_clean if i in avg_iter3_gradients], axis=0)
        refined_noisy_anchor = np.mean([avg_iter3_gradients[i] for i in refined_noisy if i in avg_iter3_gradients], axis=0)
        
        # Rescore all samples
        iter3_scores = {}
        for i in range(n_samples):
            if i not in avg_iter3_gradients:
                iter3_scores[i] = 0.0
                continue
            
            grad = avg_iter3_gradients[i]
            sim_clean = np.dot(grad, refined_clean_anchor) / (np.linalg.norm(grad) * np.linalg.norm(refined_clean_anchor) + 1e-8)
            sim_noisy = np.dot(grad, refined_noisy_anchor) / (np.linalg.norm(grad) * np.linalg.norm(refined_noisy_anchor) + 1e-8)
            iter3_scores[i] = float(sim_clean - sim_noisy)
        
        scores_list_iter3 = [iter3_scores[i] for i in range(n_samples)]
        weights_iter3 = compute_soft_weights(scores_list_iter3, GGH_MIN_WEIGHT, GGH_TEMPERATURE)
        
        # Multiply weights (iterative refinement)
        for i in range(n_samples):
            sample_weights_dict[i] = sample_weights_dict[i] * weights_iter3[i]
        
        # Normalize to [GGH_MIN_WEIGHT, 1.0]
        max_weight = max(sample_weights_dict.values())
        if max_weight > 0:
            for i in range(n_samples):
                sample_weights_dict[i] = GGH_MIN_WEIGHT + (sample_weights_dict[i] / max_weight) * (1 - GGH_MIN_WEIGHT)
    
    avg_weight_iter3 = np.mean(list(sample_weights_dict.values()))
    print(f"    Avg weight after refinement: {avg_weight_iter3:.3f}")
    
    # =============================================================================
    # DETECTION: Threshold on final weights
    # =============================================================================
    print("  Detection: Thresholding weights...")
    
    # Method 1: Fixed threshold
    detected_noisy_indices = [i for i in range(n_samples) if sample_weights_dict[i] < GGH_NOISE_THRESHOLD]
    
    # Method 2: Percentage-based (if we expect specific noise rate)
    # sorted_by_weight = sorted(range(n_samples), key=lambda i: sample_weights_dict[i])
    # n_expected_noisy = int(n_samples * DATA_NOISE_PERC)
    # detected_noisy_indices = sorted_by_weight[:n_expected_noisy]
    
    print(f"    Detected {len(detected_noisy_indices)} noisy samples (threshold={GGH_NOISE_THRESHOLD})")
    
    # =============================================================================
    # RETRAIN on cleaned data
    # =============================================================================
    print("  Retraining on cleaned data...")
    df_cleaned = DO.df_train_noisy.drop(index=detected_noisy_indices).reset_index(drop=True)
    
    DO_clean, TVM_clean, AM_clean, model_clean = full_info_run_existing_DO(
        DO, df_cleaned, r_state, num_epochs=GGH_FINAL_EPOCHS
    )
    
    # Evaluate detection (labels used only here for evaluation)
    true_noisy_indices = DO.df_train_noisy[DO.df_train_noisy['label'] == 1].index.tolist()
    detection_metrics = evaluate_detection(detected_noisy_indices, true_noisy_indices, n_samples)
    
    # Test performance
    model_clean.eval()
    with torch.no_grad():
        test_inputs, test_targets = DO.get_test_tensors(use_info="full info")
        test_preds = model_clean(test_inputs)
        ss_res = torch.sum((test_targets - test_preds) ** 2).item()
        ss_tot = torch.sum((test_targets - test_targets.mean()) ** 2).item()
        test_r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0.0
        test_mse = torch.nn.functional.mse_loss(test_preds, test_targets).item()
        test_mae = torch.nn.functional.l1_loss(test_preds, test_targets).item()
    
    return {
        'test_r2': test_r2,
        'test_mse': test_mse,
        'test_mae': test_mae,
        'detection': detection_metrics,
        'n_detected': len(detected_noisy_indices),
        'sample_weights': sample_weights_dict,
    }

print("New GGH (Unsupervised) function defined.")

## Benchmark: Compare All Methods

Run 15 trials comparing:
1. Full Info (no noise) - oracle
2. Full Info Noisy (with noise, no removal) - baseline
3. Old GGH (DBSCAN)
4. New GGH (Unsupervised Soft Refinement)

In [ ]:
# =============================================================================
# BENCHMARK EXECUTION
# =============================================================================
print("=" * 80)
print("BENCHMARK: Old GGH vs New GGH for Noise Detection")
print("=" * 80)
print(f"Dataset: Photoredox Yield")
print(f"Noise: {DATA_NOISE_PERC*100}% of data, range [{NOISE_MINRANGE}, {NOISE_MAXRANGE}]")
print(f"Runs: {BENCHMARK_N_RUNS}")
print("=" * 80)

all_results = {
    'Full Info': [],
    'Full Info Noisy': [],
    'Old GGH': [],
    'New GGH': [],
}

for run_idx in range(BENCHMARK_N_RUNS):
    r_state = run_idx
    print(f"\n{'='*60}")
    print(f"RUN {run_idx + 1}/{BENCHMARK_N_RUNS} (r_state={r_state})")
    print(f"{'='*60}")
    
    # === Create DO with noise ===
    set_to_deterministic(r_state)
    DO = DataOperator(data_path, inpt_vars, target_vars, miss_vars, hypothesis,
                      partial_perc, r_state, device="cpu", use_case="noise detection")
    DO.simulate_noise(DATA_NOISE_PERC, NOISE_MINRANGE, NOISE_MAXRANGE)
    
    true_noisy_indices = DO.df_train_noisy[DO.df_train_noisy['label'] == 1].index.tolist()
    n_total = len(DO.df_train_noisy)
    print(f"True noisy samples: {len(true_noisy_indices)}/{n_total} ({len(true_noisy_indices)/n_total*100:.1f}%)")
    
    # === Full Info (No Noise) ===
    print(\"\\nFull Info (no noise)...\")
    set_to_deterministic(r_state)
    DO_full = DataOperator(data_path, inpt_vars, target_vars, miss_vars, hypothesis,
                           partial_perc, r_state, device=\"cpu\", use_case=\"noise detection\")
    AM_full = AlgoModulators(DO_full, lr=lr)
    dataloader_full = DO_full.prep_dataloader(\"full info\", batch_size)
    model_full = initialize_model(DO_full, dataloader_full, hidden_size, r_state, dropout=dropout)
    TVM_full = TrainValidationManager(\"full info\", FULL_INFO_EPOCHS, dataloader_full, batch_size,
                                     r_state, results_path, select_gradients=True, final_analysis=False)
    TVM_full.train_model(DO_full, AM_full, model_full, final_analysis=False)
    
    model_full.eval()
    with torch.no_grad():
        test_inputs, test_targets = DO_full.get_test_tensors(use_info=\"full info\")
        test_preds = model_full(test_inputs)
        ss_res = torch.sum((test_targets - test_preds) ** 2).item()
        ss_tot = torch.sum((test_targets - test_targets.mean()) ** 2).item()
        full_r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0.0
    
    all_results['Full Info'].append({'test_r2': full_r2})
    print(f\"  R2: {full_r2:.4f}\")
    
    # === Full Info Noisy (Baseline) ===
    print(\"\\nFull Info Noisy (baseline)...\")
    set_to_deterministic(r_state)
    AM_noisy = AlgoModulators(DO, lr=lr)
    dataloader_noisy = DO.prep_dataloader(\"full info noisy\", batch_size)
    model_noisy = initialize_model(DO, dataloader_noisy, hidden_size, r_state, dropout=dropout)
    TVM_noisy = TrainValidationManager(\"full info noisy\", NOISY_EPOCHS, dataloader_noisy, batch_size,
                                      r_state, results_path, select_gradients=True, final_analysis=False)
    TVM_noisy.train_model(DO, AM_noisy, model_noisy, final_analysis=False)
    
    model_noisy.eval()
    with torch.no_grad():
        test_inputs, test_targets = DO.get_test_tensors(use_info=\"full info\")
        test_preds = model_noisy(test_inputs)
        ss_res = torch.sum((test_targets - test_preds) ** 2).item()
        ss_tot = torch.sum((test_targets - test_targets.mean()) ** 2).item()
        noisy_r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0.0
    
    all_results['Full Info Noisy'].append({'test_r2': noisy_r2})
    print(f\"  R2: {noisy_r2:.4f}\")
    
    # === Old GGH (DBSCAN) ===
    print(\"\\nOld GGH (DBSCAN)...\")
    old_result = run_old_ggh_dbscan(DO, r_state)
    all_results['Old GGH'].append(old_result)
    print(f\"  R2: {old_result['test_r2']:.4f}\")
    print(f\"  Detection - Precision: {old_result['detection']['precision']:.3f}, Recall: {old_result['detection']['recall']:.3f}\")
    
    # === New GGH (Unsupervised) ===
    print(\"\\nNew GGH (Unsupervised)...\")
    new_result = run_new_ggh_unsupervised(DO, r_state)
    all_results['New GGH'].append(new_result)
    print(f\"  R2: {new_result['test_r2']:.4f}\")
    print(f\"  Detection - Precision: {new_result['detection']['precision']:.3f}, Recall: {new_result['detection']['recall']:.3f}\")
    
    # Run summary
    print(f\"\\n>>> Summary for Run {run_idx + 1}:\")
    print(f\"    Full Info: {full_r2:.4f}\")
    print(f\"    Full Noisy: {noisy_r2:.4f}\")
    print(f\"    Old GGH: {old_result['test_r2']:.4f} (vs noisy: {old_result['test_r2'] - noisy_r2:+.4f})\")
    print(f\"    New GGH: {new_result['test_r2']:.4f} (vs noisy: {new_result['test_r2'] - noisy_r2:+.4f})\")

print(f\"\\n{'='*80}\")
print(\"BENCHMARK COMPLETE\")
print(f\"{'='*80}\")"


## Results Summary & Visualization

Statistical comparison and visualizations.

In [ ]:
# =============================================================================
# SUMMARY STATISTICS
# =============================================================================
print("\\n" + "=" * 80)
print("SUMMARY STATISTICS")
print("=" * 80)

# Extract R2 scores
full_r2_list = [r['test_r2'] for r in all_results['Full Info']]
noisy_r2_list = [r['test_r2'] for r in all_results['Full Info Noisy']]
old_r2_list = [r['test_r2'] for r in all_results['Old GGH']]
new_r2_list = [r['test_r2'] for r in all_results['New GGH']]

# R2 statistics
print(f\"\\nTest R2 Score:\")
print(f\"  Full Info (Oracle):    {np.mean(full_r2_list):.4f} ± {np.std(full_r2_list):.4f}\")
print(f\"  Full Noisy (Baseline): {np.mean(noisy_r2_list):.4f} ± {np.std(noisy_r2_list):.4f}\")
print(f\"  Old GGH (DBSCAN):      {np.mean(old_r2_list):.4f} ± {np.std(old_r2_list):.4f}\")
print(f\"  New GGH (Soft Ref):    {np.mean(new_r2_list):.4f} ± {np.std(new_r2_list):.4f}\")

# Detection statistics
old_precision_list = [r['detection']['precision'] for r in all_results['Old GGH']]
old_recall_list = [r['detection']['recall'] for r in all_results['Old GGH']]
new_precision_list = [r['detection']['precision'] for r in all_results['New GGH']]
new_recall_list = [r['detection']['recall'] for r in all_results['New GGH']]

print(f\"\\nDetection Metrics:\")
print(f\"  Old GGH - Precision: {np.mean(old_precision_list):.3f} ± {np.std(old_precision_list):.3f}\")
print(f\"  Old GGH - Recall:    {np.mean(old_recall_list):.3f} ± {np.std(old_recall_list):.3f}\")
print(f\"  New GGH - Precision: {np.mean(new_precision_list):.3f} ± {np.std(new_precision_list):.3f}\")
print(f\"  New GGH - Recall:    {np.mean(new_recall_list):.3f} ± {np.std(new_recall_list):.3f}\")

# Statistical tests
print(f\"\\n\" + \"=\" * 80)
print(\"STATISTICAL TESTS (Paired t-test)\")
print(\"=\" * 80)

# New GGH vs Old GGH
t_stat, p_val = stats.ttest_rel(new_r2_list, old_r2_list)
diff = np.mean(new_r2_list) - np.mean(old_r2_list)
sig = '***' if p_val < 0.001 else '**' if p_val < 0.01 else '*' if p_val < 0.05 else ''
print(f\"\\nNew GGH vs Old GGH:\")
print(f\"  Difference: {diff:+.4f}\")
print(f\"  t={t_stat:.3f}, p={p_val:.6f} {sig}\")
if diff > 0 and p_val < 0.05:
    print(f\"  >>> New GGH SIGNIFICANTLY OUTPERFORMS Old GGH\")
elif diff < 0 and p_val < 0.05:
    print(f\"  >>> Old GGH significantly outperforms New GGH\")
else:
    print(f\"  >>> No significant difference\")

# New GGH vs Noisy Baseline
t_stat2, p_val2 = stats.ttest_rel(new_r2_list, noisy_r2_list)
diff2 = np.mean(new_r2_list) - np.mean(noisy_r2_list)
sig2 = '***' if p_val2 < 0.001 else '**' if p_val2 < 0.01 else '*' if p_val2 < 0.05 else ''
print(f\"\\nNew GGH vs Noisy Baseline:\")
print(f\"  Difference: {diff2:+.4f}\")
print(f\"  t={t_stat2:.3f}, p={p_val2:.6f} {sig2}\")

# =============================================================================
# VISUALIZATIONS
# =============================================================================
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: R2 Comparison
ax1 = axes[0, 0]
methods = ['Full Info', 'New GGH', 'Old GGH', 'Noisy']
r2_means = [np.mean(full_r2_list), np.mean(new_r2_list), np.mean(old_r2_list), np.mean(noisy_r2_list)]
r2_stds = [np.std(full_r2_list), np.std(new_r2_list), np.std(old_r2_list), np.std(noisy_r2_list)]
colors = ['#2ecc71', '#3498db', '#9b59b6', '#e74c3c']

bars = ax1.bar(range(len(methods)), r2_means, yerr=r2_stds, capsize=5, color=colors, 
               edgecolor='black', linewidth=1.2)
ax1.set_ylabel('Test R2 Score', fontsize=12)
ax1.set_title('Test R2 Score Comparison', fontsize=14, fontweight='bold')
ax1.set_xticks(range(len(methods)))
ax1.set_xticklabels(methods, fontsize=11)
ax1.grid(axis='y', alpha=0.3)
for bar, val in zip(bars, r2_means):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, f'{val:.3f}',
             ha='center', va='bottom', fontsize=10, fontweight='bold')

# Plot 2: Per-run R2 Comparison (New vs Old GGH)
ax2 = axes[0, 1]
x = np.arange(BENCHMARK_N_RUNS)
width = 0.35
ax2.bar(x - width/2, old_r2_list, width, label='Old GGH', color='#9b59b6', alpha=0.7)
ax2.bar(x + width/2, new_r2_list, width, label='New GGH', color='#3498db', alpha=0.7)
ax2.set_xlabel('Run')
ax2.set_ylabel('Test R2')
ax2.set_title('Per-Run R2 Comparison')
ax2.legend()
ax2.set_xticks(x)
ax2.set_xticklabels([str(i+1) for i in range(BENCHMARK_N_RUNS)])

# Plot 3: Detection Precision & Recall
ax3 = axes[1, 0]
x_pos = np.arange(2)
width = 0.35
precision_means = [np.mean(old_precision_list), np.mean(new_precision_list)]
recall_means = [np.mean(old_recall_list), np.mean(new_recall_list)]
ax3.bar(x_pos - width/2, precision_means, width, label='Precision', color='#2ecc71', alpha=0.7)
ax3.bar(x_pos + width/2, recall_means, width, label='Recall', color='#e74c3c', alpha=0.7)
ax3.set_ylabel('Score')
ax3.set_title('Detection Metrics Comparison')
ax3.set_xticks(x_pos)
ax3.set_xticklabels(['Old GGH', 'New GGH'])
ax3.legend()
ax3.grid(axis='y', alpha=0.3)
for i, (prec, rec) in enumerate(zip(precision_means, recall_means)):
    ax3.text(i - width/2, prec + 0.02, f'{prec:.2f}', ha='center', fontsize=10, fontweight='bold')
    ax3.text(i + width/2, rec + 0.02, f'{rec:.2f}', ha='center', fontsize=10, fontweight='bold')

# Plot 4: Improvement Distribution
ax4 = axes[1, 1]
improvements_vs_old = [new - old for new, old in zip(new_r2_list, old_r2_list)]
improvements_vs_noisy = [new - noisy for new, noisy in zip(new_r2_list, noisy_r2_list)]
ax4.hist(improvements_vs_old, bins=10, alpha=0.6, label='New vs Old GGH', color='#3498db')
ax4.hist(improvements_vs_noisy, bins=10, alpha=0.6, label='New vs Noisy', color='#e74c3c')
ax4.axvline(x=0, color='black', linestyle='--', linewidth=1)
ax4.axvline(x=np.mean(improvements_vs_old), color='#3498db', linestyle='--', linewidth=2)
ax4.axvline(x=np.mean(improvements_vs_noisy), color='#e74c3c', linestyle='--', linewidth=2)
ax4.set_xlabel('R2 Improvement')
ax4.set_ylabel('Frequency')
ax4.set_title('Improvement Distribution')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{results_path}/benchmark_results.png', dpi=150, bbox_inches='tight')
plt.show()

# =============================================================================
# FINAL CONCLUSION
# =============================================================================
print(f\"\\n\" + \"=\" * 80)
print(\"FINAL CONCLUSION\")
print(f\"=\" * 80)
print(f\"\\nExpected (from old notebooks):\")
print(f\"  Full Info: R2 ~0.858\")
print(f\"  Noisy: R2 ~0.608\")
print(f\"  Old GGH: R2 ~0.674 (improvement of 0.066)\")
print(f\"\\nActual Results:\")
print(f\"  Full Info: R2 {np.mean(full_r2_list):.4f}\")
print(f\"  Noisy: R2 {np.mean(noisy_r2_list):.4f}\")
print(f\"  Old GGH: R2 {np.mean(old_r2_list):.4f} (improvement of {np.mean(old_r2_list) - np.mean(noisy_r2_list):+.4f})\")
print(f\"  New GGH: R2 {np.mean(new_r2_list):.4f} (improvement of {np.mean(new_r2_list) - np.mean(noisy_r2_list):+.4f})\")
print(f\"\\n>>> New GGH vs Old GGH: {diff:+.4f} (p={p_val:.4f})\")

if np.mean(new_r2_list) > 0.70:
    print(f\"\\n✓ SUCCESS: New GGH exceeds target R2 > 0.70\")
else:
    print(f\"\\n✗ Target not met: R2 {np.mean(new_r2_list):.4f} < 0.70\")

print(f\"\\n\" + \"=\" * 80)"
